In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def gradient_boosting_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    use_cv=False,
    cv_params=None,
    cv=5
):
    """
    Gradient Boosting regression for time series forecasting with:
    - Sequential boosting with decision trees
    - Time-series aware cross-validation
    - Early stopping (via n_iter_no_change)
    - Feature importance analysis
    """
    # 1. Directory setup
    os.makedirs('GradientBoostingRegression', exist_ok=True)
    
    # 2. Stationarity handling
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 3. Create lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 4. Temporal split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 5. Model training with temporal CV
    if use_cv:
        param_grid = cv_params or {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5],
            'subsample': [0.8, 1.0],
            'min_samples_split': [2, 5]
        }
        tscv = TimeSeriesSplit(n_splits=cv)
        grid_search = GridSearchCV(
            GradientBoostingRegressor(
                random_state=random_state,
                n_iter_no_change=10,
                validation_fraction=0.1
            ),
            param_grid,
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        model = GradientBoostingRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=random_state,
            n_iter_no_change=10,
            validation_fraction=0.1
        )
        model.fit(X_train, y_train)
        best_params = {
            'n_estimators': model.n_estimators_,
            'learning_rate': learning_rate,
            'max_depth': max_depth
        }

    # 6. Forecasting
    pred_test = model.predict(X_test)

    # 7. Inverse differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 8. Save forecasts
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test
    })
    forecast_csv_path = os.path.join('GradientBoostingRegression', 'gb_forecasts.csv')
    forecast_df.to_csv(forecast_csv_path, mode='a', header=not os.path.exists(forecast_csv_path), index=False)

    # 9. Calculate metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'best_params': best_params,
        'differencing': d,
        'best_iteration': model.n_estimators_,
        'feature_importances': model.feature_importances_.tolist()
    }
    metrics_df = pd.DataFrame([metrics_data])
    metrics_csv_path = os.path.join('GradientBoostingRegression', 'gb_metrics.csv')
    metrics_df.to_csv(metrics_csv_path, mode='a', header=not os.path.exists(metrics_csv_path), index=False)

    # 10. Visualization
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast'], label='GB Forecast', color='darkblue')
    plt.title(f'Gradient Boosting Forecast for {district_name}\nRMSE: {rmse:.2f}, Trees: {model.n_estimators_}')
    plt.legend()
    plt.savefig(os.path.join('GradientBoostingRegression', f'gb_forecast_{district_name}.png'))
    plt.close()

    # Feature importance plot
    features = X_train.columns
    importances = model.feature_importances_
    plt.figure(figsize=(10, 6))
    plt.barh(features, importances, color='darkblue')
    plt.title(f'Feature Importances - {district_name}')
    plt.savefig(os.path.join('GradientBoostingRegression', f'gb_importances_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'feature_importances': dict(zip(features, importances)),
        'training_stages': model.train_score_
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I40"].asfreq('MS')
        
        results = gradient_boosting_regression_lags_only(
            ts,
            district,
            use_cv=True,
            cv_params={
                'learning_rate': [0.01, 0.05, 0.1],
                'max_depth': [3, 5, 7],
                'subsample': [0.8, 1.0]
            },
            cv=3
        )

        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district       rmse  r_squared  \
0  AHMEDNAGAR  18.675721   0.938732   

                                         best_params  differencing  \
0  {'learning_rate': 0.05, 'max_depth': 7, 'subsa...             0   

   best_iteration                                feature_importances  
0              39  [0.4303565710013704, 0.36402323539313897, 0.20...  

=== Forecast Data ===
     district       date  actual   forecast
0  AHMEDNAGAR 2020-07-01    40.0  36.839105
1  AHMEDNAGAR 2020-08-01    55.0  39.622852
2  AHMEDNAGAR 2020-09-01    36.0  56.610761
3  AHMEDNAGAR 2020-10-01    82.0  40.702963
4  AHMEDNAGAR 2020-11-01    49.0  59.654072
=== Metrics ===
  district      rmse  r_squared  \
0    AKOLA  15.13538   0.931721   

                                         best_params  differencing  \
0  {'learning_rate': 0.1, 'max_depth': 7, 'subsam...             0   

   best_iteration                                feature_importances  
0              38  [0.65160389919204

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual   forecast
 0  MUMBAI SUBURBAN 2021-02-01      28  42.922635
 1  MUMBAI SUBURBAN 2021-03-01      28  37.919083,
 'metrics_df':           district       rmse  r_squared  \
 0  MUMBAI SUBURBAN  12.670305   0.985368   
 
                                          best_params  differencing  \
 0  {'learning_rate': 0.05, 'max_depth': 3, 'subsa...             1   
 
    best_iteration                                feature_importances  
 0             100  [0.8514478713671312, 0.06603140052496843, 0.08...  ,
 'feature_importances': {'lag_1': 0.8514478713671312,
  'lag_2': 0.06603140052496843,
  'lag_3': 0.08252072810790041},
 'training_stages': array([2.61219600e+02, 2.35750689e+02, 2.12764997e+02, 1.92020410e+02,
        1.73298420e+02, 1.56401824e+02, 1.41152646e+02, 1.27390263e+02,
        1.14969712e+02, 1.03760165e+02, 9.36435493e+01, 8.45133032e+01,
        7.62732561e+01, 6.88366137e+01, 6.21250438e+0